In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pyexcel_ods import save_data
import io
from collections import OrderedDict

In [ ]:
start_url = "https://consultation.avocat.fr/avocats/ville-paris.html"
data = []
def scrape_page(url):
    r = requests.get(url)

    soup_content = BeautifulSoup(r.text, 'html.parser')
    for name in soup_content.find_all('h3', class_="name linkhover"):
        result = re.sub(rf"\b{'Me '}\b", "", name.get_text())
        data.append(result)
    
    soup_page = BeautifulSoup(r.content, "html.parser")
    next_page_link = soup_page.find("a", class_="next")
    if next_page_link is not None:
        href = next_page_link.get("href")
        scrape_page(href)
    
    return data

In [ ]:
names = scrape_page(start_url)
df = pd.read_csv(io.StringIO('\n'.join(names)), names=["Name"])
      
data = OrderedDict()
data.update({"Sheet_1": df.to_numpy().tolist()})
save_data("names.ods", data)